# INF581 Lab5: Reinforcement Learning - TD Imitation Learning

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf581-2023/master/logo.jpg" style="float: left; width: 15%" />

[INF581-2023](https://moodle.polytechnique.fr/course/view.php?id=14259) Lab session #7

2022-2023 Mohamed ALAMI

In [ ]:
# These installs are necessary for packages compatibility and visualization
!sudo apt install swig
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb
!pip install stable-baselines3[extra] pyglet==1.5.27
!pip install sb3_contrib
!pip install box2d-py
!pip install gym[box2d]==0.25


In [ ]:
import stable_baselines3
print(stable_baselines3.__version__)
import gym
print(gym.__version__)
from gym import spaces
import numpy as np
import time
import pickle as pkl
from tqdm import trange, tqdm_notebook
import matplotlib.pyplot as plt
from IPython import display
from matplotlib.pyplot import figure
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pyvirtualdisplay
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

In [ ]:
#These functions are necessary to produce videos of your agent at work

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

# Behavioral Cloning

Behavioral is the simplest approach to imitation learning. The concept is straightforwardy: An expert plays a game perfectly and the learning agent has just to act as a copy cat. For each state, it has to predict what the expert would have done. Usually the expert demonstrations are typically retrieved from recording human behaviour, but for the purpose of the exercise, we well consider fully trained RL models as expert and we will try to recover their optimal policy through behavioral cloning.

The algorithm is recalled below. 

<b>Input</b>:<br>
	$\quad\quad$ Environment and expert model<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ number of episodes $n$; number of epochs: epochs

Train the expert model and get $\pi_{\text{exp}}$ <br>
For episode in range($n$): <br>
$\quad$ Initialise environment at state $s$ <br>
$\quad$ While not done: <br>
$\quad \quad$ action $a = \pi_{exp}(s)$ <br>
$\quad \quad$  $s_{1}, done = env($a$)$ <br>
$\quad \quad$ store ($s,a$) in dictionary<br>
$\quad \quad$ $s=s_1$ <br>

Initialise model $\pi$ <br>
For $k$ in range(epochs) <br>
$\quad$ Train $\pi$ for each $(s,a)$ in dictionary such as $\pi(s)=a$



#### Cartpole

We will use The Cartpole environment as a first example. More details on this environment are available [here](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) 

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

The aim of this lab is not to train the expert so we can either consider models that are already trained on Cartpole or use Stable_baselines, a framework that allows to train several heavily tested famous algorithms in one line. More details [here](https://stable-baselines.readthedocs.io/en/master/).

You can either train the imported model (should take 10 min max) or load a trained model available on moodle

In [ ]:
#we first initialise the model by saying that we will use the PPO algorithm and use a classic
# MLP parameterisation
cartpole_expert = PPO(MlpPolicy, env, verbose=1)

#Then the model can easily be trained in just one line
#cartpole_expert.learn(total_timesteps=250000)

In [ ]:
#In order to save or load the model you need to install drive 
#For this click on the folder logo on the left and then on "install drive"
#Copy the path to your current folder here

PATH = "/content/drive/MyDrive/Lab_IRL/"

In [ ]:
#You can save or load your model easily

#cartpole_expert.save(PATH + "cartpole.zip")
cartpole_expert = PPO.load(PATH + "cartpole.zip")

The next function will help us generate a video by letting the expert play on the environment for n timesteps

In [ ]:
'''
Create gifs and render it
'''
def generate_video(env, model, n_timesteps=300, initial_state=None):
  if initial_state is None:
    obs = env.reset()
  else:
    env = env.unwrapped # to access the inner functionalities of the class
    env.state = initial_state
    obs = env.state

  figure(figsize=(8, 6), dpi=80)

  # use False con Xvfb 
  _display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
  _ = _display.start()
  #model = model.float()

  frames = []

  for t in range(n_timesteps):
    #BEGIN ANSWER
      #TO DO: Simulate n steps of model playing in the environment
    #END ANSWER

    frame = env.render(mode='rgb_array')
    frames.append(frame)
    #env.render()
    time.sleep(.025)

  anim = plot_animation(frames)
  return anim

In [ ]:
env = gym.make('CartPole-v1', new_step_api=True)
anim = generate_video(env, cartpole_expert)
HTML(anim.to_html5_video())

This function is used to create expert trajectories that will be used in our Behavior Cloning model 

In [ ]:
def generate_expert_traj(model, env, n_episodes=100):
  # Sanity check
  assert (isinstance(env.observation_space, spaces.Box) or
            isinstance(env.observation_space, spaces.Discrete)), "Observation space type not supported"

  assert (isinstance(env.action_space, spaces.Box) or
            isinstance(env.action_space, spaces.Discrete)), "Action space type not supported"

  actions = []
  observations = []
  rewards = []
  episode_returns = np.zeros((n_episodes,))
  episode_starts = []  #Index of new episodes initial states

  obs = env.reset()
  episode_starts.append(True)
  reward_sum = 0.0
  idx = 0

  for ep_idx in tqdm_notebook(range(n_episodes), desc='Epoch', leave=True):

    #BEGIN ANSWER

    #TO DO: Populate observations, actions and rewards lists
    
    #END ANSWER

  if isinstance(env.observation_space, spaces.Box):
    observations = np.concatenate(observations).reshape((-1,) + env.observation_space.shape)
  elif isinstance(env.observation_space, spaces.Discrete):
    observations = np.array(observations).reshape((-1, 1))
  
  if isinstance(env.action_space, spaces.Box):
      actions = np.concatenate(actions).reshape((-1,) + env.action_space.shape)
  elif isinstance(env.action_space, spaces.Discrete):
      actions = np.array(actions).reshape((-1, 1))
      
  rewards = np.array(rewards)
  episode_starts = np.array(episode_starts[:-1])

  assert len(observations) == len(actions)

  numpy_dict = {
        'actions': actions,
        'obs': observations,
        'rewards': rewards,
        'episode_returns': episode_returns,
        'episode_starts': episode_starts
    }

  for key, val in numpy_dict.items():
    print(key, val.shape)
    
  env.close()

  return numpy_dict

In [ ]:
#Generate expert trajectories
cartpole_demos = generate_expert_traj(cartpole_expert, env)

In [ ]:
'''
Define BC Model as NN

Specs:
NN: 3 layers, the middle layer should be of size (32,64)
Loss: choose it wisely
'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
    
    #BEGIN ANSWER
        #TO DO: Define network as described above
    #END ANSWER

    def forward(self, x):
      #BEGIN ANSWER
    	  #TO DO: Define forward funcion and activations
      #END ANSWER
    	return x

In [ ]:
#This function transforms the dataset of expert state-actions pairs into a Dataset class
#That makes it easier to train in batches

class NumpyDataset(data.Dataset):

    def __init__(self, array, transform=None):
        super().__init__()
        self.obs = torch.FloatTensor(array["obs"])
        self.actions = torch.FloatTensor(array["actions"])
        self.transform = transform

    def __len__(self):
        return len(self.obs)

    def __getitem__(self, index):
        obs = self.obs[index]
        action = self.actions[index]
        if self.transform:
            obs = self.transform(obs)
            action = self.transform(action)
        return obs, action


In [ ]:
loader_args = dict(batch_size=128, shuffle=True)
train_dset = NumpyDataset(cartpole_demos)
train_loader = data.DataLoader(train_dset, **loader_args)
device = "cpu"

In [ ]:
def plot_train_curves(epochs, train_losses, test_losses, title=''):
    x = np.linspace(0, epochs, len(train_losses))
    plt.figure()
    plt.plot(x, train_losses, label='train_loss')
    #if test_losses:
        #plt.plot(x, test_losses, label='test_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.show()
  
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0

    for obs, action in train_loader:


      # TO DO: Implement the training loop that is done in an epoch. This function is called in the function train_epochs below



      return avg_loss.item()

def train_epochs(model, train_loader, criterion, test_loader=None, train_args=None, plot=True):
    # training parameters
    epochs, lr = train_args['epochs'], train_args['lr']
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses, test_losses = [], []
    for epoch in tqdm_notebook(range(epochs), desc='Epoch', leave=False):
        model.train()
        train_loss = train(model, train_loader, criterion, optimizer)
        train_losses.append(train_loss)
           
    if plot:
        plot_train_curves(epochs, train_losses, test_losses, title='Training Curve')
    return train_losses, test_losses

In [ ]:

#TO DO: Choose the right criterion
#criterion = ...


cartpole_BC_model = Net()
train_args = dict(epochs=3000, lr=0.0001)
train_losses, test_losses = train_epochs(cartpole_BC_model, train_loader, criterion, train_args=train_args)

In [ ]:
anim = generate_video(env, cartpole_BC_model)
HTML(anim.to_html5_video())

Even with a very quick training, we see that the agent is pretty good. But is it that good ? 

Now let's try with a different initial state

In [ ]:
obs = env.reset()
state_init = np.array([1.0, obs[1], obs[2], obs[3]]) #we slightly move the initial position of the cart

anim = generate_video(env, cartpole_BC_model, initial_state=state_init)
HTML(anim.to_html5_video())

When changing the initial state, we put the agent in a situation the expert has never seen; therefore the agent is lost and does not know what is th good action to perform. 

Actually it is not even necessary to change the initial state. The agent is never absolutely perfect, so with a long enough horizon, it will make some mistakes that stack up and drive it into states that the expert has never seen (as it only explores best states).  

### Lunar Lander

Let's test with the Lunar Lander environment

In [ ]:
env = gym.make('LunarLander-v2')

#model = PPO(MlpPolicy, env, verbose=1)
#model.learn(total_timesteps=250000)

In [ ]:
#model.save(PATH + "lander.zip")

In [ ]:
expert_lander = PPO.load(PATH + "lander.zip")

In [ ]:
env = gym.make('LunarLander-v2', new_step_api=True)
anim = generate_video(env, expert_lander, 400)
HTML(anim.to_html5_video())

In [ ]:
Lander_demos = generate_expert_traj(expert_lander, env)

In [ ]:
'''
Define BC Model as NN

Specs:
NN: 5 layers. Width of hidden layers: 24,64,128,256
Loss and activation functions to define
'''
class Lander_Net(nn.Module):
    def __init__(self):
        super(Lander_Net, self).__init__()
        #BEGIN ANSWER
          #TO DO: define NN as described above
        #END ANSWER

    def forward(self, x):
      #BEGIN ANSWER
      # TO DO: Define forward function and activations
      #END ANSWER
      return x

In [ ]:
loader_args = dict(batch_size=128, shuffle=True)
train_dset = NumpyDataset(Lander_demos)
train_loader = data.DataLoader(train_dset, **loader_args)

#BEGIN ANSWER
#criterion = ...
#END ANSWER

Lander_BC_model = Lander_Net()
train_args = dict(epochs=2000, lr=0.001)
train_losses, test_losses = train_epochs(Lander_BC_model, train_loader, criterion, train_args=train_args)

In [ ]:
env = gym.make('LunarLander-v2', new_step_api=True)
anim = generate_video(env, Lander_BC_model, 500)
HTML(anim.to_html5_video())

### Lunar Lander stochastic

Now let's try something else. We will add wind in the environment to make the dynamics of the environement stochastic. 

In [ ]:
env = gym.make('LunarLander-v2', enable_wind=True)
#model = PPO(MlpPolicy, env, verbose=1)
#model.learn(total_timesteps=250000)

In [ ]:
#model.save(PATH + "windy_lander.zip")

In [ ]:
expert_windy_lander = PPO.load(PATH + "windy_lander.zip")

In [ ]:
env = gym.make('LunarLander-v2', enable_wind=True, new_step_api=True)
anim = generate_video(env, expert_windy_lander, 400)
HTML(anim.to_html5_video())

In [ ]:
Lander_demos = generate_expert_traj(expert_windy_lander, env)

In [ ]:
device = "cpu"
loader_args = dict(batch_size=128, shuffle=True)
train_dset = NumpyDataset(Lander_demos)
train_loader = data.DataLoader(train_dset, **loader_args)

#BEGIN ANSWER
#criterion = ...
#END ANSWER

Windy_Lander_BC_model = Lander_Net()
train_args = dict(epochs=2000, lr=0.001)
train_losses, test_losses = train_epochs(Windy_Lander_BC_model, train_loader, criterion, train_args=train_args)

In [ ]:
anim = generate_video(env, Windy_Lander_BC_model, 500)
HTML(anim.to_html5_video())

The agent fails because stochastic environments increases "mistakes" as well as the chances to end in non optimal states that the expert never encountered

### DAgger

To mitigate that problem, a classic solution is to use DAgger algorithm (Dataset Aggregation). The idea is simple. We just have to create a feedback loop between the agent and the expert. After each roll or episode of the agent, its trajectory (the list of states) it encountered are given back to the expert, who attribute to them the right action to perform. The nex state-action pairs are then added to the original dataset, and a new training is done on the aggregated dataset

Below is the algorithm's pseudocode: <br>
<b>Input</b>:<br>
environment, expert model, agent model, number of iterations, number of trajectories to sample from the agent at each iteration <br>

<b>Algorithm</b>: <br>

Initialise dataset <br>
<b>FOR</b> $i$ in range(number of iterations): <br>
$\quad$ traj $\xleftarrow{}$generate model trajectories <br>
$\quad$ get states from traj <br>
$\quad$ get expert actions for states <br>
$\quad$ add states and actions to dataset <br>
$\quad$ train agent on dataset

In [ ]:
#function to extract expert actions from given states
def get_expert_actions(model, states):
  #BEGIN ANSWER 
  # actions = ...
  #END ANSWER
  return actions

def DAgger(env, expert, model, n_iterations=10, n_traj=10, iter_plot=1):

  dataset = {'obs': None, 'actions': None}
  losses = []
  #BEGIN ANSWER
  #criterion = ...
  #END ANSWER

  for i in tqdm_notebook(range(n_iterations)):
    #BEGIN ANSWER

    #TO DO: Dataset aggregation
    
    #END ANSWERS

    train_dset = NumpyDataset(dataset)
    train_loader = data.DataLoader(train_dset, **loader_args)
    
    train_losses, test_losses = train_epochs(model, train_loader, criterion, train_args=train_args, plot=False)

    

    if losses == []:
      losses = train_losses
    else:
      losses = np.concatenate((losses, np.array(train_losses)))
    if n_iterations % iter_plot == 0:
      plt.plot(losses)
      plt.show()   

In [ ]:
Dagger_Lander = Lander_Net()
device = "cpu"
loader_args = dict(batch_size=128, shuffle=True)


train_args = dict(epochs=100, lr=0.001)

env = gym.make('LunarLander-v2', new_step_api=True)
DAgger(env, expert_lander, Dagger_Lander)

In [ ]:
anim = generate_video(env, Dagger_Lander, 500)
HTML(anim.to_html5_video())

In [ ]:
Windy_Dagger_Lander = Lander_Net()
device = "cpu"
loader_args = dict(batch_size=128, shuffle=True)
train_args = dict(epochs=100, lr=0.001)

DAgger(env, expert_windy_lander, Windy_Dagger_Lander, n_iterations = 10, n_traj=50, iter_plot=1)

In [ ]:
anim = generate_video(env, Windy_Dagger_Lander, 500)
HTML(anim.to_html5_video())

### MaxEnt IRL

It would be great if we could avoid having the expert in the loop while making sure that different initial states or stochastic dynamics have a limited impoact on the agen performance. That is the aim of Inverse RL. Instead of learning a policy given a reward, we learn a reward given expert demonstrations. The aim is to retrieve the reward function that the expert followed and then use that learned reward to train a classic RL algorithm. 

Below is the algorithm pseudocode. 

<b>Initialise</b>:<br>
- feature_matrix as an identity matrix of dimension number_of_states <br>
- expert feature expectations as a vector of size number_of_states 
- environment to get initial state $s$<br>
- q_table $Q$<br>
- $\theta$ and learner_feature_expectations as vectors of size number_of_states ($\theta$ values are negative)<br>
- $\gamma$ the discount factor
- learning_rate
- update_freq: frequency of reward update
<b>FOR</b> episode in number_episodes: <br>
$\quad$ <b> IF </b> episode % update==0: <br>
$\quad \quad$ gradient = expert_feature_expectations - learner_feature_expectations/episode <br>
$\quad \quad$ $\theta$ += learning_rate*gradient <br>
$\quad \quad$ clip $\theta$: all values that are greater to 0 are fixed to 0 <br>
$\quad$ <b> While</b> True: <br>
$\quad \quad$ Draw $a\sim\mathcal{U}[0,1]$ <br>
$\quad \quad$ <b>IF</b>: $a<\epsilon$ <br>
$\quad \quad\quad$ Draw action randomly for action space <br>
$\quad \quad$ <b> ELSE </b>: action = $\argmax_a$ q_table[$s$] <br>
$\quad \quad$ next_state = env(action) <br>
$\quad \quad$ reward = $\theta$.$\text{feature_matrix[next_state]}^T$ <br>
$\quad \quad$ $Q(s,action) = reward+\gamma \max_aQ(next state,a)$ <br>
$\quad \quad$  learner_feature_expectations += feature_matrix(state) <br>
$\quad \quad$ <b>IF</b> done: break

In [ ]:
env = gym.make("MountainCar-v0")
#model = PPO(MlpPolicy, env, verbose=1)
#model.learn(total_timesteps=250000)

In [ ]:
#model.save(PATH + "mountain_car.zip")

In [ ]:
expert_car = PPO.load(PATH + "mountain_car.zip")

In [ ]:
env = gym.make("MountainCar-v0", new_step_api=True)
anim = generate_video(env, expert_car, 400)
HTML(anim.to_html5_video())

In [ ]:
car_demos = generate_expert_traj(expert_car, env)

In [ ]:
# The state space of mountain car is continous. As we use Q-learning we need finite state spaces. We therefore discretize it
# The state space will become an array n_bins*n_bins (one dimension per state dimension)
n_actions = env.action_space.n
n_bins = 30
n_states = n_bins**2
feature_matrix = np.eye((n_states))

#For each state, this function associates it to the corresponding state index and builds the feature vector.
#The feature vector is a vector of shape n_bins with a 1 at the state index
def feature_vector(state, env=env, bins=n_bins):
  try:
    idx = np.array(np.arange(bins**2)).reshape(bins,bins)
    bin_length = (env.observation_space.high - env.observation_space.low)/(bins-1)
    bin_id = (state - env.observation_space.low)/bin_length
    vector_id = idx[int(bin_id[0]), int(bin_id[1])]
  except:
    print(state, env.observation_space.high, env.observation_space.low,  bin_length, bin_id)
  return feature_matrix[vector_id], vector_id

#This function classifies the state-action pairs that belong to the same trajectory
def process_demos(demos):
  demonstrations = {}
  demonstrations['obs'] = []
  demonstrations["actions"] = []
  new_episodes_idx = np.where(demos["episode_starts"]==True)[0]
  for i in range(new_episodes_idx.shape[0]-1):
    demonstrations['obs'].append(demos['obs'][new_episodes_idx[i]:new_episodes_idx[i+1]])
    demonstrations['actions'].append(demos['actions'][new_episodes_idx[i]:new_episodes_idx[i+1]])
  return demonstrations

#Build expert_feature_expectations from demonstrations
def expert_feature_expectations(feature_matrix, demonstrations, n_states=n_states):
  n_demonstrations = len(demonstrations['obs'])
  feature_expectations = np.zeros((n_demonstrations,n_states))
  #BEGIN ANSWER
  #TO DO: build feature expectation for each demonstration
  #END ANSWER
  return feature_expectations

In [ ]:
demonstrations = process_demos(car_demos)
expert_expectations = expert_feature_expectations(feature_matrix, demonstrations)

In [ ]:
#This function calculates the gradient, updates theta abd clips it
def maxent_irl(expert, learner, theta, learning_rate):
  #BEGIN ANSWER
  # gradient = ...
  # theta += ...

    # Clip theta

  #END ANSWER
  return theta

#Calculates the irl_reward of a given state
def get_reward(state, theta):
  #BEGIN ANSWER
  # irl_reward = ...
  #END ANSWER
  return irl_reward

#Performs Q-Learning update
def update_q_table(q_table, state, action, reward, next_state):
  #BEGIN ANSWER
  #q_1 = ...  Q-value of state-action pair
  #q_2 = ...  Q-value of next state
  #q_table[state][action] = ... Update
  #END ANSWER
  return q_table

In [ ]:
env = gym.make("MountainCar-v0")
learner_feature_expectations = np.zeros(n_states)
theta = - np.random.uniform(size=(n_states,))
q_table = np.zeros((n_states, n_actions))
gamma = 0.99
q_learning_rate = 0.03
theta_learning_rate = 0.05
eps = 0.2

episodes, scores, mean_scores = [], [], []

for episode in tqdm_notebook(range(30000),desc="Episode", leave=True):
  state = env.reset()
  score = 0

  if episode !=0 and episode % 5000 == 0:
    learner = learner_feature_expectations / episode
    theta = maxent_irl(expert_expectations, learner, theta, theta_learning_rate)
  
  if episode !=0 and episode % 1000 == 0:
    plt.plot(mean_scores)
    plt.show()

  while True:
    _, state_id = feature_vector(state)
    a = np.random.uniform()
    if a < eps:
      action = env.action_space.sample()
    else:
      action = np.argmax(q_table[state_id])

    next_state, reward, done, _ = env.step(action)
    irl_reward = get_reward(state, theta)
    _, next_state_id = feature_vector(next_state)
    q_table = update_q_table(q_table, state_id, action, irl_reward, next_state_id)
    learner_feature_expectations += feature_matrix[int(state_id)]
    score += reward
    state = next_state

    if done:
      scores.append(score)
      episodes.append(episode)
      mean_scores.append(np.mean(scores))
      break


In [ ]:
def generate_car_video(env, q_table, n_timesteps=300):
  obs = env.reset()
  figure(figsize=(8, 6), dpi=80)

  # use False con Xvfb 
  _display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
  _ = _display.start()

  frames = []
  for t in range(n_timesteps):
    state_id = feature_vector(obs)[1]
    action = np.argmax(q_table[state_id])
    obs, rewards, done, info = env.step(action)
    frame = env.render(mode='rgb_array')
    frames.append(frame)
    time.sleep(.025)

  anim = plot_animation(frames)
  return anim

In [ ]:
anim = generate_car_video(env, q_table)
HTML(anim.to_html5_video())

We were able to solve mountain car using our learned reward. A final remark is that each time the reward is updates, the MDP has to be solved with the new reward. Solving an RL problem is usually hard enough so solving it multiple times is nearly computationally  infeasible for environment that are bigger than mountain car. 

Getting rid of that constraint is one of the main research topic on this subject. Several approaches solved it. Those interested can have a look at:
- [GAIL](https://arxiv.org/abs/1606.03476)
- [Guided Cost Learning](https://arxiv.org/abs/1603.00448)